In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', None)

import spacy
from spacy.training import Example
import random

import ahocorasick
spacy.prefer_gpu()

True

In [2]:
# df = pd.read_csv("./Address Elements Extraction Dataset/train.csv")

# df.set_index("id", inplace=True)

# df['POI'] = np.nan
# df['street'] = np.nan
df = pd.read_csv('./split_result.csv')

In [3]:
df_test = pd.read_csv("./Address Elements Extraction Dataset/test.csv")
df_test.set_index("id", inplace=True)

In [4]:
df.head()

,id,raw_address,POI/street,POI,street
0,0,jl kapuk timur delta sili iii lippo cika 11 a cicau cikarang pusat,/jl kapuk timur delta sili iii lippo cika,NaN,jl kapuk timur delta sili iii lippo cika
1,1,"aye, jati sampurna",/,NaN,NaN
2,2,setu siung 119 rt 5 1 13880 cipayung,/siung,NaN,siung
3,3,"toko dita, kertosono",toko dita/,toko dita,NaN
4,4,jl. orde baru,/jl. orde baru,NaN,jl. orde baru


In [5]:
df_test.head()

,raw_address
id,
0,s. par 53 sidanegara 4 cilacap tengah
1,"angg per, baloi indah kel. lubuk baja"
2,"asma laun, mand imog,"
3,"ud agung rej, raya nga sri wedari karanganyar"
4,"cut mutia, 35 baiturrahman"


In [6]:
# model_name = "./mix_model"
# model_name = "./filter_poi"
# # model_name = "./street_model"
# model_name = "./filter_model"
model_name = "./filter_model_2"

In [14]:
spacy.require_gpu()
nlp_model = spacy.load(model_name)
poi_model = spacy.load("./filter_poi")
street_model = spacy.load("./filter_street")

In [15]:
count = 0
for idx, row in df.iloc[100:].iterrows():
#     if  isinstance(row['POI'], float):
    count += 1
    print(f"address: {row['raw_address']}")
    print(f"expected poi: {row['POI']}")
    print(f"expected street: {row['street']}\n")
    doc_filter = nlp_model(row['raw_address'])
    for ent in doc_filter.ents:
        print("\t **", ent.text, "-", ent.label_)
        
    doc_poi = poi_model(row['raw_address'])
    for ent in doc_poi.ents:
        print("\t ", ent.text, "-", ent.label_)
    
    doc_street = street_model(row['raw_address'])
    for ent in doc_street.ents:
        print("\t ", ent.text, "-", ent.label_)

    print("-" * 50)
    if count == 10:
        break

address: kedai tenun jep senn, kota bumi, kebon melati
expected poi: kedai tenun jepara sennaart
expected street: kota bumi

	  kedai tenun - POI
	  kota bumi - STREET
--------------------------------------------------
address: wadungasri dalam waru raya wad asri, 24 sidoarjo
expected poi: dalam waru
expected street: raya wad asri

	 ** wadungasri dalam waru - POI
	 ** raya wad asri - STREET
	  wadungasri dalam waru - POI
	  raya wad asri - STREET
--------------------------------------------------
address: bulusan tim barat iii, no 35 3 tembalang
expected poi: nan
expected street: tim barat iii

	  bulusan tim barat iii - STREET
--------------------------------------------------
address: bakti jaya bukit perm vii 8 15315 setu
expected poi: nan
expected street: bukit perm vii

	 ** bukit perm vii - STREET
	  bakti jaya - POI
	  bukit perm vii - STREET
--------------------------------------------------
address: jl terusan buah batu no 185. samping indomaret. bandung.
expected poi: sampin

In [16]:
submission = []
for idx, row in df_test.iloc[:].iterrows():
    
#     doc_filter = nlp_model(row['raw_address'])
    doc_street = street_model(row['raw_address'])
    doc_poi = poi_model(row['raw_address'])
    
    tmp = {'id': idx}
    for ent in doc_street.ents:
        tmp["Smodel"+ent.label_] = ent.text
    for ent in doc_poi.ents:
        tmp["Pmodel"+ent.label_] = ent.text
        
    submission.append(tmp)

In [30]:
submission_mix = []
for idx, row in df_test.iloc[:].iterrows():
    
    doc_filter = nlp_model(row['raw_address'])
    
    tmp = {'id': idx}
    for ent in doc_filter.ents:
        tmp[ent.label_] = ent.text
        
    submission_mix.append(tmp)

In [31]:
submission =pd.DataFrame(submission_mix)
submission['raw_address'] = df_test['raw_address']

In [32]:
submission.columns = ['id', 'STREET', 'POI', 'raw_address']

In [33]:
submission.head(20)

,id,STREET,POI,raw_address
0,0,s. par,sidanegara 4,s. par 53 sidanegara 4 cilacap tengah
1,1,angg per,NaN,"angg per, baloi indah kel. lubuk baja"
2,2,mand imog,NaN,"asma laun, mand imog,"
3,3,raya nga sri,NaN,"ud agung rej, raya nga sri wedari karanganyar"
4,4,cut mutia,NaN,"cut mutia, 35 baiturrahman"
5,5,NaN,NaN,pem dos dapur ala perum gar no a 12 suka jaya sukarami
6,6,NaN,NaN,"tb. mara, cisayong"
7,7,taman beji tista,NaN,"pura taman beji tista,"
8,8,NaN,NaN,"mangkura tk pam, 90113 ujung pandang"
9,9,raya won,NaN,raya won wonotunggal wonotunggal


In [72]:
submission['POI'] = final_p

In [34]:
submission = submission.fillna("")
combine_lambda = lambda x: '{}/{}'.format(x['POI'], x['STREET'])
submission["POI/street"] = submission.apply(combine_lambda, axis = 1)

In [35]:
submission = pd.DataFrame({'id':submission['id'],
                  'POI/street':submission['POI/street']})
submission.to_csv('./submission_{}.csv'.format('filter2'), header=True, index=False)

In [36]:
submission.head(20)

,id,POI/street
0,0,sidanegara 4/s. par
1,1,/angg per
2,2,/mand imog
3,3,/raya nga sri
4,4,/cut mutia
5,5,/
6,6,/
7,7,/taman beji tista
8,8,/
9,9,/raya won


In [38]:
filter2_df = pd.read_csv("./submission_filter2.csv")
mixed_df = pd.read_csv("./submission_SPmodel.csv")

filter2_df[['POI', 'street']] = filter2_df['POI/street'].str.split('/', 1, expand=True)
filter2_df['POI/street'].replace('/', np.nan, inplace=True)
filter2_df['POI'].replace('', np.nan, inplace=True)
filter2_df['street'].replace('', np.nan, inplace=True)


mixed_df[['POI', 'street']] = mixed_df['POI/street'].str.split('/', 1, expand=True)
mixed_df['POI/street'].replace('/', np.nan, inplace=True)
mixed_df['POI'].replace('', np.nan, inplace=True)
mixed_df['street'].replace('', np.nan, inplace=True)
mixed_df.head()

,id,POI/street,POI,street
0,0,/s. par,NaN,s. par
1,1,baloi indah kel./angg per,baloi indah kel.,angg per
2,2,asma laun/mand imog,asma laun,mand imog
3,3,ud agung rej/raya nga sri,ud agung rej,raya nga sri
4,4,/cut mutia,NaN,cut mutia


In [39]:
filter2_poi = filter2_df['POI'].to_list()
filter2_street = filter2_df['street'].to_list()
mixed_poi = mixed_df['POI'].to_list()
mixed_street = mixed_df['street'].to_list()

In [49]:
print(len(filter2_poi))
print(len(filter2_street))
print(len(mixed_poi))
print(len(mixed_street))

50000
50000
50000
50000


In [57]:
ans_df = filter2_df.copy()

In [58]:
ans_df.head()

,id,POI/street,POI,street
0,0,sidanegara 4/s. par,sidanegara 4,s. par
1,1,/angg per,NaN,angg per
2,2,/mand imog,NaN,mand imog
3,3,/raya nga sri,NaN,raya nga sri
4,4,/cut mutia,NaN,cut mutia


In [59]:
ans_df['POI'] = mixed_poi

In [60]:
ans_df.head()

,id,POI/street,POI,street
0,0,sidanegara 4/s. par,NaN,s. par
1,1,/angg per,baloi indah kel.,angg per
2,2,/mand imog,asma laun,mand imog
3,3,/raya nga sri,ud agung rej,raya nga sri
4,4,/cut mutia,NaN,cut mutia


In [61]:
ans_df = ans_df.fillna("")
combine_lambda = lambda x: '{}/{}'.format(x['POI'], x['street'])
ans_df["POI/street"] = ans_df.apply(combine_lambda, axis = 1)

ans_df = pd.DataFrame({'id':ans_df['id'],
                  'POI/street':ans_df['POI/street']})

ans_df.to_csv('./submission_{}.csv'.format('filterAndPmodel'), header=True, index=False)

In [62]:
ans_df.head()

,id,POI/street
0,0,/s. par
1,1,baloi indah kel./angg per
2,2,asma laun/mand imog
3,3,ud agung rej/raya nga sri
4,4,/cut mutia
